Transplant train/validate/test model from Hex to VS Code

In [1]:
# ============================================
# Environment setup (local execution)
# ============================================

import numpy as np
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import joblib
import json

In [2]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 200)

In [3]:
# Only need train data for this workflow
train_data = pd.read_csv("../data/raw/train.csv")

# Quick sanity checks
print("Rows, cols:", train_data.shape)
train_data.head()

Rows, cols: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y = train_data['Survived']
X = train_data.drop(columns=['Survived'])

In [5]:
# Define a function to perform the data transformations
def transform_data(data):
    data = data.copy()
    
    # Drop 'Name' column
    data.drop(columns=['Name'], inplace=True)
    
    # Convert 'Sex' to 0 for 'male' and 1 for 'female'
    sex_mapping = {'male': 0, 'female': 1}
    data['Sex'] = data['Sex'].map(sex_mapping)

    # Encode Age as a binary "present vs missing" indicator (unchanged logic)
    data['Age'] = data['Age'].fillna(0)
    data['Age'] = data['Age'].apply(lambda x: 1 if x > 0 else x)
    
    # Rename for correct semantics (NO value change)
    data.rename(columns={'Age': 'AgeMissing'}, inplace=True)

    # Create a new feature 'FamSz' by summing 'SibSp' and 'Parch'
    data['FamSz'] = data['SibSp'] + data['Parch']

    # Drop 'Ticket', 'Cabin', 'Fare', 'PassengerId' columns
    data.drop(columns=['Ticket', 'Cabin', 'Fare', 'PassengerId'], inplace=True)

    # Fill missing values in 'Embarked' with mode
    embarked_mode = data['Embarked'].mode()[0]
    data['Embarked'] = data['Embarked'].fillna(embarked_mode)

    # Convert 'Embarked' to numeric values (1, 2, 3)
    embarked_mapping = {'S': 1, 'C': 2, 'Q': 3}
    data['Embarked'] = data['Embarked'].map(embarked_mapping)
    
    return data

In [6]:
y = train_data["Survived"]
X = transform_data(train_data.drop(columns=["Survived"]))

In [7]:
feature_names = X.columns.tolist()
print(feature_names)

with open("../artifacts/feature_names.json", "w") as f:
    json.dump(feature_names, f, indent=2)

['Pclass', 'Sex', 'AgeMissing', 'SibSp', 'Parch', 'Embarked', 'FamSz']


In [8]:
print("X shape:", X.shape)
print("Missing values:", X.isnull().sum().sum())
X.head()

X shape: (891, 7)
Missing values: 0


,Pclass,Sex,AgeMissing,SibSp,Parch,Embarked,FamSz
0,3,0,1.0,1,0,1,1
1,1,1,1.0,1,0,2,1
2,3,1,1.0,0,0,1,0
3,1,1,1.0,1,0,1,1
4,3,0,1.0,0,0,1,0


In [9]:
from sklearn.model_selection import StratifiedKFold

RANDOM_STATE = 42
cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("rf", RandomForestClassifier(random_state=42))
    ]
)

param_grid = {
    "rf__n_estimators": [2, 4, 8, 10, 25, 50, 100],
    "rf__max_depth": [None, 2, 3, 4, 8, 16],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf": [1, 2, 4, 6],
    "rf__bootstrap": [True, False],
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv10,
    n_jobs=-1,
    verbose=2
)

In [11]:
# ============================================
# Run GridSearchCV
# ============================================

grid_search.fit(X, y)

Fitting 10 folds for each of 1008 candidates, totalling 10080 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'rf__bootstrap': [True, False], 'rf__max_depth': [None, 2, ...], 'rf__min_samples_leaf': [1, 2, ...], 'rf__min_samples_split': [2, 5, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1

In [12]:
# ============================================
# Inspect GridSearch results
# ============================================

print("Best CV score:", grid_search.best_score_)
print("Best parameters:")
for k, v in grid_search.best_params_.items():
    print(f"  {k}: {v}")

Best CV score: 0.8136454431960051
Best parameters:
  rf__bootstrap: True
  rf__max_depth: 3
  rf__min_samples_leaf: 1
  rf__min_samples_split: 2
  rf__n_estimators: 25


In [13]:
# ============================================
# Final hyperparameters (OOF-selected)
# ============================================

FINAL_RF_PARAMS = {
    "bootstrap": True,
    "max_depth": 3,
    "min_samples_leaf": 1,
    "min_samples_split": 2,
    "n_estimators": 25,
    "random_state": 42
}

FINAL_RF_PARAMS

{'bootstrap': True,
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 25,
 'random_state': 42}

In [14]:
# ============================================
# Final scaler fit (100% of data)
# ============================================

from sklearn.preprocessing import StandardScaler

final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(X)

# Wrap back into DataFrame to preserve feature names
X_scaled_df = pd.DataFrame(
    X_scaled,
    columns=X.columns,
    index=X.index
)

X_scaled_df.head()

,Pclass,Sex,AgeMissing,SibSp,Parch,Embarked,FamSz
0,0.827377,-0.737695,0.497895,0.432793,-0.473674,-0.568837,0.059160
1,-1.566107,1.355574,0.497895,0.432793,-0.473674,1.005181,0.059160
2,0.827377,1.355574,0.497895,-0.474545,-0.473674,-0.568837,-0.560975
3,-1.566107,1.355574,0.497895,0.432793,-0.473674,-0.568837,0.059160
4,0.827377,-0.737695,0.497895,-0.474545,-0.473674,-0.568837,-0.560975


In [15]:
# ============================================
# Final model fit (100% of data)
# ============================================

from sklearn.ensemble import RandomForestClassifier

final_model = RandomForestClassifier(**FINAL_RF_PARAMS)
final_model.fit(X_scaled_df, y)

final_model

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",25
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(y_t

In [16]:
# ============================================
# Export artifacts for Hex
# ============================================

import joblib
import json
import os

os.makedirs("artifacts", exist_ok=True)

# Model + scaler
joblib.dump(final_model, "../artifacts/model.joblib")
joblib.dump(final_scaler, "../artifacts/scaler.joblib")

# Feature names (contract)
with open("../artifacts/feature_names.json", "w") as f:
    json.dump(list(X.columns), f, indent=2)

print("Artifacts written:")
print(" - artifacts/model.joblib")
print(" - artifacts/scaler.joblib")
print(" - artifacts/feature_names.json")

Artifacts written:
 - artifacts/model.joblib
 - artifacts/scaler.joblib
 - artifacts/feature_names.json


In [17]:
# ============================================
# Sanity check: reload artifacts
# ============================================

loaded_model = joblib.load("../artifacts/model.joblib")
loaded_scaler = joblib.load("../artifacts/scaler.joblib")

# Pick a few rows
X_sample = X.iloc[:5]
X_sample_scaled = pd.DataFrame(
    loaded_scaler.transform(X_sample),
    columns=X_sample.columns,
    index=X_sample.index
)

probs = loaded_model.predict_proba(X_sample_scaled)[:, 1]
probs

array([0.26465134, 0.83295516, 0.48551084, 0.8162008 , 0.18669138])

Further artifacts for Hex

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

eval_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("rf", RandomForestClassifier(**FINAL_RF_PARAMS)),
    ]
)

In [19]:
import numpy as np

oof_pred = np.zeros(len(X), dtype=int)

In [20]:
from sklearn.metrics import accuracy_score

for fold, (train_idx, val_idx) in enumerate(cv10.split(X, y), start=1):
    print(f"Fold {fold}")

    X_train_fold = X.iloc[train_idx]
    y_train_fold = y.iloc[train_idx]

    X_val_fold = X.iloc[val_idx]

    # Fit pipeline on training fold only
    eval_pipeline.fit(X_train_fold, y_train_fold)

    # Predict labels for validation fold
    oof_pred[val_idx] = eval_pipeline.predict(X_val_fold)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10


In [21]:
from sklearn.metrics import accuracy_score

oof_accuracy = accuracy_score(y, oof_pred)
oof_accuracy

0.813692480359147

In [22]:
# ============================================
# Export offline evaluation metrics for Hex
# ============================================

import json
import os

OOF_ACCURACY = oof_accuracy

metrics = {
    "oof_accuracy": OOF_ACCURACY,
    "oof_accuracy_display": f"{OOF_ACCURACY:.2f}",
    "evaluation_type": "offline",
    "evaluation_method": "10-fold stratified cross-validation (OOF)",
    "note": (
        "Accuracy computed offline using out-of-fold predictions from a "
        "StandardScaler + RandomForest pipeline. "
        "This metric is provided for context and is not recomputed in the app."
    )
}

os.makedirs("../artifacts", exist_ok=True)

with open("../artifacts/metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print("Wrote ../artifacts/metrics.json")
print(metrics)

Wrote ../artifacts/metrics.json
{'oof_accuracy': 0.813692480359147, 'oof_accuracy_display': '0.81', 'evaluation_type': 'offline', 'evaluation_method': '10-fold stratified cross-validation (OOF)', 'note': 'Accuracy computed offline using out-of-fold predictions from a StandardScaler + RandomForest pipeline. This metric is provided for context and is not recomputed in the app.'}


In [23]:
# ----------------------------------
# OOF loop
# ----------------------------------
for train_idx, val_idx in cv10.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train = y.iloc[train_idx]

    pipeline.fit(X_train, y_train)
    oof_pred[val_idx] = pipeline.predict(X_val)


In [24]:
# ----------------------------------
# Evaluation dataframe
# ----------------------------------
eval_df = X.copy()

eval_df["y_true"] = y.values
eval_df["y_pred"] = oof_pred
eval_df["Error"] = eval_df["y_true"] != eval_df["y_pred"]

# Human-readable labels
eval_df["Sex"] = eval_df["Sex"].map({0: "Male", 1: "Female"})
eval_df["Passenger class"] = eval_df["Pclass"].map({
    1: "1st class",
    2: "2nd class",
    3: "3rd class"
})

In [25]:
# ----------------------------------
# Prediction error rates by cohort
# ----------------------------------
prediction_error_rates = (
    eval_df
    .groupby(["Sex", "Passenger class"])
    .agg(
        Number_of_passengers=("Error", "count"),
        Prediction_Error_Rate=("Error", "mean")
    )
    .reset_index()
)

prediction_error_rates

,Sex,Passenger class,Number_of_passengers,Prediction_Error_Rate
0,Female,1st class,94,0.053191
1,Female,2nd class,76,0.078947
2,Female,3rd class,144,0.402778
3,Male,1st class,122,0.377049
4,Male,2nd class,108,0.175926
5,Male,3rd class,347,0.152738


In [26]:
import os

os.makedirs("../artifacts", exist_ok=True)

output_path = "../artifacts/prediction_error_rates.csv"
prediction_error_rates.to_csv(output_path, index=False)

print(f"Artifact written: {output_path}")

Artifact written: ../artifacts/prediction_error_rates.csv


In [27]:
import pandas as pd

# Build base dataframe
gt_df = X.copy()
gt_df["y_true"] = y

# Human-readable fields
gt_df["Sex"] = gt_df["Sex"].map({0: "Male", 1: "Female"})
gt_df["Passenger class"] = gt_df["Pclass"].map({
    1: "1st class",
    2: "2nd class",
    3: "3rd class"
})

# Aggregate ground truth by cohort
ground_truth = (
    gt_df
    .groupby(["Sex", "Passenger class"])
    .agg(
        Total_Passengers=("y_true", "count"),
        Survivors=("y_true", "sum")
    )
    .reset_index()
)

# Derived numeric fields
ground_truth["Non_survivors"] = (
    ground_truth["Total_Passengers"] - ground_truth["Survivors"]
)

ground_truth["Survival_Rate"] = (
    ground_truth["Survivors"] / ground_truth["Total_Passengers"]
)

ground_truth

,Sex,Passenger class,Total_Passengers,Survivors,Non_survivors,Survival_Rate
0,Female,1st class,94,91,3,0.968085
1,Female,2nd class,76,70,6,0.921053
2,Female,3rd class,144,72,72,0.500000
3,Male,1st class,122,45,77,0.368852
4,Male,2nd class,108,17,91,0.157407
5,Male,3rd class,347,47,300,0.135447


In [28]:
ground_truth.to_csv(
    "../artifacts/ground_truth_by_cohort.csv",
    index=False
)

In [29]:
import numpy as np

oof_pred = np.zeros(len(X), dtype=int)
oof_pred_proba = np.zeros(len(X), dtype=float)

In [30]:
from sklearn.metrics import accuracy_score

for fold, (train_idx, val_idx) in enumerate(cv10.split(X, y), start=1):
    print(f"Fold {fold}")

    X_train_fold = X.iloc[train_idx]
    y_train_fold = y.iloc[train_idx]

    X_val_fold = X.iloc[val_idx]

    # Fit pipeline on training fold only
    eval_pipeline.fit(X_train_fold, y_train_fold)

    # OOF predictions
    oof_pred[val_idx] = eval_pipeline.predict(X_val_fold)
    oof_pred_proba[val_idx] = eval_pipeline.predict_proba(X_val_fold)[:, 1]

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10


In [31]:
import pandas as pd

evaluation_df = X.copy()

evaluation_df["y_true"] = y.values
evaluation_df["y_pred"] = oof_pred
evaluation_df["Survival_Probability"] = oof_pred_proba

evaluation_df.head()

,Pclass,Sex,AgeMissing,SibSp,Parch,Embarked,FamSz,y_true,y_pred,Survival_Probability
0,3,0,1.0,1,0,1,1,0,0,0.244212
1,1,1,1.0,1,0,2,1,1,1,0.828324
2,3,1,1.0,0,0,1,0,1,1,0.507703
3,1,1,1.0,1,0,1,1,1,1,0.784260
4,3,0,1.0,0,0,1,0,0,0,0.175389


In [32]:
# Shape check
assert evaluation_df.shape[0] == X.shape[0]

# No missing probabilities
assert not evaluation_df["Survival_Probability"].isnull().any()

# Probability bounds
assert evaluation_df["Survival_Probability"].between(0, 1).all()

evaluation_df.dtypes

Pclass                    int64
Sex                       int64
AgeMissing              float64
SibSp                     int64
Parch                     int64
Embarked                  int64
FamSz                     int64
y_true                    int64
y_pred                    int64
Survival_Probability    float64
dtype: object

In [33]:
import os

os.makedirs("../artifacts", exist_ok=True)

output_path = "../artifacts/evaluation_universe.csv"
evaluation_df.to_csv(output_path, index=False)

print(f"Artifact written: {output_path}")
print(f"Rows: {evaluation_df.shape[0]}")
print(f"Columns: {evaluation_df.shape[1]}")

Artifact written: ../artifacts/evaluation_universe.csv
Rows: 891
Columns: 10


In [ ]:
# ============================================
# Kaggle Titanic submission for your Hex-a-thon model
# Uses: artifacts/model.joblib + artifacts/scaler.joblib + artifacts/feature_names.json
# Input: ../data/raw/test.csv
# Output: ../submissions/submission.csv
# ============================================

import os
import json
import joblib
import pandas as pd
import numpy as np

# ----------------------------
# Paths (adjust if needed)
# ----------------------------
ARTIFACT_DIR = "../artifacts"
DATA_DIR = "../data/raw"
OUT_DIR = "../submissions"

MODEL_PATH = os.path.join(ARTIFACT_DIR, "model.joblib")
SCALER_PATH = os.path.join(ARTIFACT_DIR, "scaler.joblib")
FEATURES_PATH = os.path.join(ARTIFACT_DIR, "feature_names.json")

TEST_PATH = os.path.join(DATA_DIR, "test.csv")
OUT_PATH = os.path.join(OUT_DIR, "submission.csv")

os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------
# Load artifacts
# ----------------------------
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

with open(FEATURES_PATH, "r") as f:
    feature_names = json.load(f)

print("Loaded artifacts:")
print(" - model:", type(model))
print(" - scaler:", type(scaler))
print(" - feature_names:", feature_names)

# ----------------------------
# Feature engineering (MUST match training)
# ----------------------------
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Drop 'Name'
    if "Name" in df.columns:
        df.drop(columns=["Name"], inplace=True)

    # Sex: male->0, female->1
    if "Sex" in df.columns:
        df["Sex"] = df["Sex"].map({"male": 0, "female": 1}).astype("Int64")

    # AgeMissing: 0 if missing, 1 if present (your earlier logic)
    # In your earlier code: fillna(0) then 1 if x>0 else x (so 0 means missing)
    if "Age" in df.columns:
        age_filled = df["Age"].fillna(0)
        df["AgeMissing"] = (age_filled > 0).astype(int)
        df.drop(columns=["Age"], inplace=True)
    else:
        # If Age is absent for some reason, default to missing
        df["AgeMissing"] = 0

    # FamSz
    if "SibSp" in df.columns and "Parch" in df.columns:
        df["FamSz"] = df["SibSp"].fillna(0).astype(int) + df["Parch"].fillna(0).astype(int)
    else:
        df["FamSz"] = 0

    # Drop columns not used
    for col in ["Ticket", "Cabin", "Fare"]:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

    # Embarked: fill with mode (from *test*; acceptable for submission)
    # If you want strict reproducibility, store train-mode in an artifact and use it here.
    if "Embarked" in df.columns:
        if df["Embarked"].isna().any():
            mode_val = df["Embarked"].mode(dropna=True)
            fill_val = mode_val.iloc[0] if len(mode_val) else "S"
            df["Embarked"] = df["Embarked"].fillna(fill_val)

        df["Embarked"] = df["Embarked"].map({"S": 1, "C": 2, "Q": 3}).astype("Int64")
    else:
        df["Embarked"] = 1  # default to S if missing

    # Ensure integer columns are plain int (after potential Int64 NA type)
    for c in ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "AgeMissing", "FamSz"]:
        if c in df.columns:
            df[c] = df[c].fillna(0).astype(int)

    return df

# ----------------------------
# Load test data
# ----------------------------
test_raw = pd.read_csv(TEST_PATH)
if "PassengerId" not in test_raw.columns:
    raise ValueError("test.csv must include PassengerId")

passenger_ids = test_raw["PassengerId"].copy()

# Transform
X_test = transform_data(test_raw)

# Keep ONLY the expected features, in the expected order
missing = [c for c in feature_names if c not in X_test.columns]
extra = [c for c in X_test.columns if c not in feature_names]

if missing:
    raise ValueError(f"Missing expected feature columns in transformed test data: {missing}")

# Reindex to contract
X_test = X_test.reindex(columns=feature_names)

# ----------------------------
# Scale + predict
# ----------------------------
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=feature_names,
    index=X_test.index
)

# Probability of survival (class 1)
proba = model.predict_proba(X_test_scaled)[:, 1]

# Default threshold 0.5 (standard Kaggle baseline)
y_pred = (proba >= 0.5).astype(int)

# ----------------------------
# Build submission
# ----------------------------
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": y_pred
})

submission.to_csv(OUT_PATH, index=False)
print(f"Submission written: {OUT_PATH}")
print(submission.head())


Loaded artifacts:
 - model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
 - scaler: <class 'sklearn.preprocessing._data.StandardScaler'>
 - feature_names: ['Pclass', 'Sex', 'AgeMissing', 'SibSp', 'Parch', 'Embarked', 'FamSz']
Submission written: ../submissions\submission.csv
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1


c:\Users\agbro\OneDrive\Desktop\Jupyter - VS Code\hex-a-thon\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
